In [1]:
import glob
import os
import pandas as pd
from datetime import date, timedelta
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"
dts_path = "/Users/User/Downloads/Datasets/"

# Directory to be monitored
directory = '/Users/User/Downloads'
# Dictionary of file categories and their extensions
categories = {
    'Images': ['jpeg', 'jpg', 'png'],
    'PDFs': ['pdf'],
    'Datasets': ['csv', 'xlsx', 'xls', 'json'],
    'Videos': ['mp4','m4a'],
    'Zips': ['zip'],
    'EXEs': ['exe', 'msi'],
    'Pythons': ['ipynb'],
}
pd.options.display.max_rows = 11
pd.options.display.float_format = '{:.2f}'.format
year = 2023
month = 6
today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 6, 23), datetime.date(2023, 6, 22))

In [2]:
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2023-06-23
yesterday: 2023-06-22


### Restart and Run All Cells

## Must rerun whenever there is change in orders

In [3]:
sql = """
SELECT trade, name, qty, price AS target, active, reason, market, xdate 
FROM orders
ORDER BY trade, name"""
orders = pd.read_sql(sql, conlite)
orders.shape

(36, 8)

In [4]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
input_file

'../data/Price-Today.csv'

In [5]:
prices = pd.read_csv(input_file)
prices.shape

(192, 10)

In [6]:
df_merge  = pd.merge(orders, prices, on = 'name', how='inner')
df_merge.shape

(36, 17)

In [7]:
df_merge['spd'] = 0

In [8]:
colo = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'price', 'change', '%change',
        'active', 'xdate']
df_merge[colo].shape

(36, 12)

In [9]:
file_name = "Price-for-Order.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

df_merge[colo].to_csv(output_file, header=True, index=False)
df_merge[colo].to_csv(osd_file, header=True, index=False)

### call ruby ruby\order-log-new.rb

In [10]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [11]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [12]:
!ruby ruby\\order-log-new.rb

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
B,BANPU,-27,RD30pct,SET50,3000,7.8,9.15,0.05,0.549451,0,2023-04-10
B,BBL,-76,,SET50,200,125.0,163.0,0.0,0.0,0,2023-04-21
B,BDMS,-23,XXX,SET50,1000,24.0,28.25,-0.25,-0.877193,0,2022-12-22
B,BEM,-10,52WL,SET50,2500,7.8,8.3,-0.1,-1.190476,0,2022-12-22
B,BH,-90,5pct,SET50,200,169.0,228.0,-4.0,-1.724138,0,2022-12-22
B,CPNREIT,-8,RD40pct,SET,3000,10.2,11.0,0.6,5.769231,2,2023-05-23
B,EA,-8,52WL,SET50,1200,56.0,58.0,1.0,1.754386,1,2022-12-22
B,KKP,-8,,SET100,500,58.0,60.0,0.25,0.41841,1,2023-04-28
B,KTB,-51,,SET50,1500,14.3,19.4,-0.1,-0.512821,0,2023-04-20
B,LANNA,-17,,SET,1500,13.8,15.5,0.4,2.649007,0,2023-03-18
B,MCS,-1,RD55pct,SET,3000,6.7,6.75,0.0,0.0,1,2023-03-17
B,NER,-6,RD40pct,SET,5000,4.54,4.66,-0.04,-0.851064,1,2023-04-20
B,ORI,-6,52WL,SET100,3000,10.0,10.6,-0.1,-0.934579,1,2023-05-08
B,PTG,-2,,SET100,1800,11.4,11.6,-0.1,-0.854701,1,2022-12-22
B,PTTEP,-29,,SET50,200,134.5,149.0,0.0,0.0,0,2023-02-14
B,SINGER,

In [13]:
os.chdir("C:\\users\\user\\onedrive\\a7\\daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\daily'

In [14]:
file_name   = 'orders-log.csv'
input_file = data_path + file_name

df = pd.read_csv(input_file, sep=',', index_col=None)
df.query('spd >= -3 and spd <= 3')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
10,B,MCS,-1,RD55pct,SET,3000,6.70,6.75,0.00,0.00,1,2023-03-17
13,B,PTG,-2,NaN,SET100,1800,11.40,11.60,-0.10,-0.85,1,2022-12-22
16,B,STA,-1,RD55pct,SET100,1200,17.00,17.10,-0.50,-2.84,1,2023-04-19
17,B,SYNEX,-1,RND55pct,SET,3000,12.50,12.60,-0.10,-0.79,1,2023-03-09
19,B,TFFIF,-3,DOS,SET,10000,7.55,7.70,0.10,1.32,1,2023-05-31


In [15]:
df.query('active == 2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
5,B,CPNREIT,-8,RD40pct,SET,3000,10.20,11.00,0.60,5.77,2,2023-05-23
25,S,DIF,5,5pct,SET,2000,11.10,10.60,-0.20,-1.85,2,2022-02-02
26,S,GVREIT,13,5pct,SET,10000,9.10,8.45,-0.10,-1.17,2,2023-05-26
35,S,WHART,4,5pct,SET,10000,11.00,10.60,0.10,0.95,2,2022-02-02


In [16]:
df.query('active == 1')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
6,B,EA,-8,52WL,SET50,1200,56.00,58.00,1.00,1.75,1,2022-12-22
7,B,KKP,-8,NaN,SET100,500,58.00,60.00,0.25,0.42,1,2023-04-28
10,B,MCS,-1,RD55pct,SET,3000,6.70,6.75,0.00,0.00,1,2023-03-17
11,B,NER,-6,RD40pct,SET,5000,4.54,4.66,-0.04,-0.85,1,2023-04-20
12,B,ORI,-6,52WL,SET100,3000,10.00,10.60,-0.10,-0.93,1,2023-05-08
...,...,...,...,...,...,...,...,...,...,...,...,...
19,B,TFFIF,-3,DOS,SET,10000,7.55,7.70,0.10,1.32,1,2023-05-31
20,B,TMT,-8,RD30pct,SET,3000,6.95,7.35,0.00,0.00,1,2023-04-18
27,S,ICHI,9,5pct,SET,1500,15.00,14.10,0.00,0.00,1,2022-12-22
29,S,JASIF,8,CUT,SET,10000,7.50,7.10,-0.05,-0.70,1,2023-05-19


In [17]:
df.query('active == 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
0,B,BANPU,-27,RD30pct,SET50,3000,7.80,9.15,0.05,0.55,0,2023-04-10
1,B,BBL,-76,NaN,SET50,200,125.00,163.00,0.00,0.00,0,2023-04-21
2,B,BDMS,-23,XXX,SET50,1000,24.00,28.25,-0.25,-0.88,0,2022-12-22
3,B,BEM,-10,52WL,SET50,2500,7.80,8.30,-0.10,-1.19,0,2022-12-22
4,B,BH,-90,5pct,SET50,200,169.00,228.00,-4.00,-1.72,0,2022-12-22
...,...,...,...,...,...,...,...,...,...,...,...,...
28,S,IVL,14,5pct,SET50,2400,37.75,34.25,0.00,0.00,0,2023-05-26
30,S,JMART,46,5pct,SET50,1200,20.60,16.00,-0.10,-0.62,0,2023-04-11
31,S,JMT,20,5pct,SET50,1800,41.75,36.75,-0.25,-0.68,0,2023-04-11
33,S,PTTGC,64,RD45pct,SET50,600,53.25,37.25,0.00,0.00,0,2023-02-27


In [18]:
colx = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'current', 'change', '%change',
        'active', 'xdate']

file_name = "orders-log.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

df[colx].to_csv(output_file, header=True, index=False)
df[colx].to_csv(osd_file, header=True, index=False)

### Convert column name to make it valid for df.query

In [19]:
df['percent'] = df['%change']

In [20]:
df.query('percent < -2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,percent
16,B,STA,-1,RD55pct,SET100,1200,17.00,17.10,-0.50,-2.84,1,2023-04-19,-2.84
22,S,AH,27,5pct,SET,1200,39.00,32.25,-1.00,-3.01,0,2022-12-22,-3.01


In [21]:
df.query('percent >= 2.00')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,percent
5,B,CPNREIT,-8,RD40pct,SET,3000,10.20,11.00,0.60,5.77,2,2023-05-23,5.77
9,B,LANNA,-17,NaN,SET,1500,13.80,15.50,0.40,2.65,0,2023-03-18,2.65


In [22]:
df.loc[df.trade == 'B'].sort_values('spd',ascending=False)

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,percent
10,B,MCS,-1,RD55pct,SET,3000,6.70,6.75,0.00,0.00,1,2023-03-17,0.00
17,B,SYNEX,-1,RND55pct,SET,3000,12.50,12.60,-0.10,-0.79,1,2023-03-09,-0.79
16,B,STA,-1,RD55pct,SET100,1200,17.00,17.10,-0.50,-2.84,1,2023-04-19,-2.84
13,B,PTG,-2,NaN,SET100,1800,11.40,11.60,-0.10,-0.85,1,2022-12-22,-0.85
19,B,TFFIF,-3,DOS,SET,10000,7.55,7.70,0.10,1.32,1,2023-05-31,1.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,B,PTTEP,-29,NaN,SET50,200,134.50,149.00,0.00,0.00,0,2023-02-14,0.00
21,B,TTB,-49,52WL,SET50,20000,1.09,1.58,-0.01,-0.63,0,2023-04-18,-0.63
8,B,KTB,-51,NaN,SET50,1500,14.30,19.40,-0.10,-0.51,0,2023-04-20,-0.51
1,B,BBL,-76,NaN,SET50,200,125.00,163.00,0.00,0.00,0,2023-04-21,0.00


### Sell recently purchased stocks if profitable

In [23]:
pd.set_option('display.max_rows', 40)
sql = """SELECT name, qty, price AS cost, date 
FROM buys B 
JOIN stocks S 
ON stock_id = S.id 
WHERE status = 'Active'
ORDER BY name, date"""
buys = pd.read_sql(sql, conpf)
buys.shape

(111, 4)

In [24]:
colu = 'name date_x qty cost price difa %change %profit'.split()
buys_price = pd.merge(buys, prices, on='name', how='inner')
buys_price['diff'] = buys_price['price'] - buys_price['cost']
buys_price['difa'] = buys_price['diff'] * buys_price['qty']
buys_price['%profit'] = round(buys_price['difa'] / (buys_price['qty'] * buys_price['cost']) * 100,2)

In [25]:
laggards = buys_price.loc[buys_price.cost > buys_price.price,colu]
laggards.shape

(108, 8)

In [26]:
watch_list = buys_price.loc[buys_price.cost <= buys_price.price,colu]
watch_list.sort_values('name',ascending='False')

,name,date_x,qty,cost,price,difa,%change,%profit
21,DIF,2023-06-23,2000,10.60,10.60,0.00,-1.85,0.00
50,KTC,2023-06-15,900,50.00,50.50,450.00,0.50,1.00
110,WHART,2022-05-17,10000,10.50,10.60,1000.00,0.95,0.95


### Profit per Day

In [27]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s AND MONTH(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % (year, month)
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2023 AND MONTH(sells.date) = 6
ORDER BY sells.date DESC, name


In [28]:
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days
0,CPNREIT,2023,6,10.70,10.20,0.50,"3,000","32,100.00","30,600.00","1,500.00",4.90,3
1,TFFIF,2023,6,7.70,7.65,0.05,"10,000","77,000.00","76,500.00",500.00,0.65,17
2,TFFIF,2023,6,7.70,7.65,0.05,"10,000","77,000.00","76,500.00",500.00,0.65,28
3,NER,2023,6,4.84,4.60,0.24,"5,000","24,200.00","23,000.00","1,200.00",5.22,15
4,JMT,2023,6,41.75,39.50,2.25,600,"25,050.00","23,700.00","1,350.00",5.70,3
5,ORI,2023,6,11.20,10.20,1.00,"3,000","33,600.00","30,600.00","3,000.00",9.80,23
6,EA,2023,6,64.00,61.00,3.00,400,"25,600.00","24,400.00","1,200.00",4.92,7
7,KTC,2023,6,52.50,50.00,2.50,400,"21,000.00","20,000.00","1,000.00",5.00,1


In [29]:
file_name = "monthly-sales.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

sells_df.to_csv(output_file, header=True, index=False)
sells_df.to_csv(osd_file, header=True, index=False)

In [30]:
sells_df['profit_per_day'] = (sells_df['gross'] / sells_df['buy_amt']) / sells_df['days'] * 100
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days,profit_per_day
0,CPNREIT,2023,6,10.70,10.20,0.50,"3,000","32,100.00","30,600.00","1,500.00",4.90,3,1.633987
1,TFFIF,2023,6,7.70,7.65,0.05,"10,000","77,000.00","76,500.00",500.00,0.65,17,0.038447
2,TFFIF,2023,6,7.70,7.65,0.05,"10,000","77,000.00","76,500.00",500.00,0.65,28,0.023343
3,NER,2023,6,4.84,4.60,0.24,"5,000","24,200.00","23,000.00","1,200.00",5.22,15,0.347826
4,JMT,2023,6,41.75,39.50,2.25,600,"25,050.00","23,700.00","1,350.00",5.70,3,1.898734
5,ORI,2023,6,11.20,10.20,1.00,"3,000","33,600.00","30,600.00","3,000.00",9.80,23,0.426257
6,EA,2023,6,64.00,61.00,3.00,400,"25,600.00","24,400.00","1,200.00",4.92,7,0.702576
7,KTC,2023,6,52.50,50.00,2.50,400,"21,000.00","20,000.00","1,000.00",5.00,1,5.000000


In [31]:
average_profit_percent = sells_df['profit_per_day'].mean()
print(f"Average Profit Percent: {average_profit_percent:.2f}%")

Average Profit Percent: 1.26%
